In [1]:
import pandas as pd
import json
from collections import Counter

## Take a look at the input data

In [2]:
input_data = 'supplier_car.json'

#read input json file, take care of the encoding!
d = [json.loads(line) for line in open(input_data, 'r', encoding='utf-8')]
df = pd.DataFrame(data=d)

In [3]:
df.shape

(21906, 9)

In [4]:
df.head()

,ID,MakeText,TypeName,TypeNameFull,ModelText,ModelTypeText,Attribute Names,Attribute Values,entity_id
0,976.0,MERCEDES-BENZ,McLaren,MERCEDES-BENZ SLR McLaren,SLR,SLR McLaren,Seats,2,0001fda6-192b-46a8-bc08-0e833f904eed
1,1059.0,MERCEDES-BENZ,ML 350 Inspiration,MERCEDES-BENZ ML 350 Inspiration,ML 350,ML 350 Inspiration,Hp,235,00107c2d-0071-4475-88f0-810133638b7e
2,524.0,AUDI,S6 Avant quattro 4.2,AUDI S6 Avant quattro 4.2,S6,S6 Avant quattro 4.2,FuelTypeText,Benzin,00126794-a8ef-48fe-93d6-43cfc69fbfb6
3,608.0,SAAB,9-3 2.0i-16 TS Aero,SAAB 9-3 2.0i-16 TS Aero,9-3,9-3 2.0i-16 TS Aero,Ccm,1985,00182529-1bf7-4f93-89fa-2e8e634b2c9d
4,726.0,PORSCHE,911 Turbo Cabrio,PORSCHE 911 Turbo Cabrio,911,911 Turbo Cabrio,BodyColorText,schwarz mét.,002d30c2-43f6-4905-868f-160dbc445c56


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21906 entries, 0 to 21905
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                21906 non-null  object
 1   MakeText          21906 non-null  object
 2   TypeName          21906 non-null  object
 3   TypeNameFull      21906 non-null  object
 4   ModelText         20957 non-null  object
 5   ModelTypeText     21906 non-null  object
 6   Attribute Names   21906 non-null  object
 7   Attribute Values  21906 non-null  object
 8   entity_id         21906 non-null  object
dtypes: object(9)
memory usage: 1.5+ MB


In [6]:
df.tail()

,ID,MakeText,TypeName,TypeNameFull,ModelText,ModelTypeText,Attribute Names,Attribute Values,entity_id
21901,33.0,MERCEDES-BENZ,730PS,MERCEDES-BENZ SLR 730PS,SLR,SLR 730PS,FirstRegMonth,6,feaddbdf-670d-41a5-8cb0-ad29755e85f1
21902,15.0,LAND ROVER,Range Rover 3.6 d HSE,LAND ROVER Range Rover 3.6 d HSE,RANGE ROVER,Range Rover 3.6 d HSE,ConsumptionRatingText,G,fecd4372-41bb-4fa3-9849-e218feac176c
21903,408.0,MERCEDES-BENZ,CLK 55 AMG Avantgarde,MERCEDES-BENZ CLK 55 AMG Avantgarde,CLK 55 AMG,CLK 55 AMG Avantgarde,Seats,4,ff70cfbc-d9d1-4bae-a0bd-12cab94ef94c
21904,146.0,BMW,Turbo,BMW 2002 Turbo,2002,2002 Turbo,DriveTypeText,Hinterradantrieb,ffc135a1-3332-43ae-b8f6-f07128b3c37c
21905,830.0,SMART,Fortwo passion,SMART Fortwo passion,FORTWO,Fortwo passion,Ccm,698,ffcd0c3c-e5ab-420b-8ed0-dcda67ea4017


In [7]:
#convert ID column to numeric type and sort it
df['ID'] = df['ID'].astype('float')
df = df.sort_values("ID")

## Check what the Target Data looks like

In [8]:
t_data = pd.read_excel('Target Data.xlsx')
t_data.head()

,carType,color,condition,currency,drive,city,country,make,manufacture_year,mileage,mileage_unit,model,model_variant,price_on_request,type,zip,manufacture_month,fuel_consumption_unit
0,Convertible / Roadster,White,Used,USD,LHD,Zuzwil,CH,Toyota,1983,0.0,kilometer,Celica,2800 i Supra,False,car,NaN,3.0,l_km_consumption
1,Convertible / Roadster,Other,Original Condition,USD,LHD,London,GB,Ferrari,2001,0.0,mile,550,NaN,True,car,NaN,NaN,NaN
2,Convertible / Roadster,White,Used,USD,LHD,Scotts Valley,US,Porsche,1989,0.0,mile,"911 ""G""",NaN,True,car,NaN,NaN,NaN
3,Convertible / Roadster,Blue,Used,USD,LHD,London,GB,Rolls-Royce,1961,0.0,kilometer,Silver Cloud II,NaN,False,car,NaN,NaN,NaN
4,Convertible / Roadster,Black,Original Condition,USD,LHD,Hong Kong,HK,Gumpert,2010,0.0,kilometer,Apollo,Apollo S 800hp,False,car,NaN,NaN,NaN


### Have a look at the Attribute Names 

In [9]:
pd.DataFrame(Counter(df["Attribute Names"]), index=["Attribute Names"])

,Km,InteriorColorText,BodyColorText,Properties,ConsumptionTotalText,Hp,Ccm,BodyTypeText,ConsumptionRatingText,Doors,Seats,ConditionTypeText,TransmissionTypeText,FuelTypeText,FirstRegYear,City,FirstRegMonth,Co2EmissionText,DriveTypeText
Attribute Names,1153,1153,1153,1153,1153,1153,1153,1152,1153,1153,1153,1153,1153,1153,1153,1153,1153,1153,1153


## Pre-processing

In [10]:
# I need to transform the supplier data to achieve the same granularity as the target data

In [11]:
# Some rows in the input data have repeting ID and column values, while only the Attribute Names and Attribute Values change.
# We can group them and then make new columns with all Attribute Names and Attribute Values in single row for each ID
df_grp = df.groupby(["ID", "MakeText", "TypeName", "TypeNameFull", "ModelText", "ModelTypeText"], dropna=False)
df_grp_agg = df_grp.agg({"Attribute Names": list, "Attribute Values": list}).reset_index()

# some ModelText vaues are NaN, keeping all values by dropna=False
# the line below helps with normalization later
df_grp_agg['ModelText'] = df_grp_agg['ModelText'].astype('str')

In [12]:
# define function to aggragate attributes
def aggr_attr(row):
    attr = {key: value for key, value in zip(row["Attribute Names"], row["Attribute Values"])}
    attr.update({col_name: row[col_name] for col_name in ["ID", "MakeText", "TypeName", "TypeNameFull", "ModelText", "ModelTypeText"]})
    return pd.Series(attr)

df_grp_agg_attr = df_grp_agg.apply(aggr_attr, axis=1)

In [13]:
df_grp_agg_attr.shape

(1153, 25)

In [14]:
df_grp_agg_attr.head()

,BodyColorText,BodyTypeText,Ccm,City,Co2EmissionText,ConditionTypeText,ConsumptionRatingText,ConsumptionTotalText,Doors,DriveTypeText,...,InteriorColorText,Km,MakeText,ModelText,ModelTypeText,Properties,Seats,TransmissionTypeText,TypeName,TypeNameFull
0,anthrazit,Limousine,3199,Zuzwil,275 g/km,Occasion,null,11.5 l/100km,4,Allrad,...,grau,31900,MERCEDES-BENZ,E 320,E 320 Elégance 4-Matic,"""Ab MFK""",5,Automat,E 320 Elégance 4-Matic,MERCEDES-BENZ E 320 Elégance 4-Matic
1,anthrazit,Kombi,4991,Zuzwil,333 g/km,Occasion,G,14.0 l/100km,5,Allrad,...,null,25400,AUDI,RS6,RS6 Avant 5.0 V10 quattro,"""Ab MFK""",5,Automat sequentiell,RS6 Avant 5.0 V10 quattro,AUDI RS6 Avant 5.0 V10 quattro
2,anthrazit,Kombi,4172,Zuzwil,350 g/km,Occasion,G,14.6 l/100km,5,Allrad,...,beige,38500,AUDI,RS6,RS6 Avant quattro,"""Ab MFK""",5,Automat sequentiell,RS6 Avant quattro,AUDI RS6 Avant quattro
3,anthrazit,Coupé,6162,Zuzwil,291 g/km,Occasion,G,12.7 l/100km,2,Hinterradantrieb,...,schwarz,200,CHEVROLET,CORVETTE,Corvette Z06,"""Ab MFK"", ""Direkt-/Parallelimport""",2,Schaltgetriebe manuell,Corvette Z06,CHEVROLET Corvette Z06
4,anthrazit,SUV / Geländewagen,4806,Zuzwil,270 g/km,Occasion,G,11.5 l/100km,5,Allrad,...,schwarz,2900,PORSCHE,CAYENNE,Cayenne Turbo Techart Magnum Kit,"""Ab MFK""",5,Automat sequentiell,Cayenne Turbo Techart Magnum Kit,PORSCHE Cayenne Turbo Techart Magnum Kit


## Normalization

### Normalize column carType

In [15]:
pd.DataFrame({k: v for k, v in Counter(t_data["carType"]).most_common()}, index=["carType"])

,Coupé,Convertible / Roadster,Other,Saloon,Custom,SUV,NaN,Station Wagon,Targa,Single seater
carType,3359,1970,902,362,200,187,101,90,62,19


In [16]:
pd.DataFrame({k: v for k, v in Counter(df_grp_agg_attr["BodyTypeText"]).most_common()}, index=["BodyTypeText"])

,Coupé,Limousine,Cabriolet,Kombi,SUV / Geländewagen,Kleinwagen,Kompaktvan / Minivan,Pick-up,Wohnkabine,Sattelschlepper,NaN
BodyTypeText,369,325,248,98,89,13,5,3,1,1,1


### Check the number of seats

In [17]:
pd.DataFrame({k: v for k, v in Counter(df_grp_agg_attr["Seats"]).most_common()}, index=["Seats"])

,5,2,4,0,3,7,1,6
Seats,480,326,298,38,4,3,2,2


### Normalize color

In [18]:
pd.DataFrame({k: v for k, v in Counter(t_data["color"]).most_common()}, index=["color"])

,Other,Red,Black,Blue,Silver,White,Gray,Green,Yellow,Beige,Orange,Brown,Gold,Purple
color,1433,1207,942,840,678,671,501,454,204,99,88,75,35,25


In [19]:
pd.DataFrame({k: v for k, v in Counter(df_grp_agg_attr["BodyColorText"]).most_common()}, index=["BodyColorText"])

,schwarz mét.,silber mét.,blau mét.,grau mét.,schwarz,weiss,rot,grün mét.,blau,anthrazit mét.,...,beige mét.,bordeaux,gold mét.,orange,braun mét.,orange mét.,beige,braun,gelb mét.,gold
BodyColorText,232,208,110,102,94,63,60,48,41,33,...,4,4,4,3,2,2,1,1,1,1


### Normalize condition

In [20]:
pd.DataFrame({k: v for k, v in Counter(df_grp_agg_attr["ConditionTypeText"]).most_common()}, index=["ConditionTypeText"])

,Occasion,Oldtimer,Neu,Vorführmodell
ConditionTypeText,1098,29,17,9


In [21]:
pd.DataFrame({k: v for k, v in Counter(t_data["condition"]).most_common()}, index=["condition"])

,Used,Restored,Original Condition,Used with guarantee,New,Restoration Project
condition,5190,1005,744,162,108,43


### Normalize model and variant

In [22]:
pd.DataFrame({k: v for k, v in Counter(df_grp_agg_attr["MakeText"]).most_common()}, index=["MakeText"])

,MERCEDES-BENZ,PORSCHE,BMW,AUDI,FERRARI,RENAULT,LAMBORGHINI,BENTLEY,MASERATI,VW,...,KAISER,HARLEY-DAVIDSON,SMART,PGO,WIESMANN,DE TOMASO,DeLorean,AUTOBIANCHI,MINI,AGM
MakeText,295,137,126,118,68,47,38,35,32,19,...,1,1,1,1,1,1,1,1,1,1


In [23]:
pd.DataFrame({k: v for k, v in Counter(df_grp_agg_attr["ModelText"]).most_common()}, index=["ModelText"])

,911,RS6,nan,SL 500,CAYENNE,SL 600,SL 55 AMG,ALPINE,A8,M5,...,ML 350,PANTERA,A112,Silverado,MX-5,205,XJ 220,ESPACE,MINI,A 150
ModelText,95,58,50,33,27,21,18,18,17,16,...,1,1,1,1,1,1,1,1,1,1


In [24]:
pd.DataFrame({k: v for k, v in Counter(df_grp_agg_attr["ModelTypeText"]).most_common()}, index=["ModelTypeText"])

,RS6 Avant quattro,SL 500,911 Turbo,RS6 Avant 5.0 V10 quattro,SL 600,SL 55 AMG,Cayenne Turbo,M5,A8 4.2 quattro,CL 600,...,MX-5 1.8i 16V Confort,205 Turbo 16V,320d xDrive M-Sportpaket,XJ 220,Espace 2.0 T Privilège,1000,MP4-12C Cabriolet 3.8 V8,WLR,A 150 Classic,Reventon Coupé
ModelTypeText,31,30,30,23,21,18,18,16,15,12,...,1,1,1,1,1,1,1,1,1,1


In [25]:
pd.DataFrame({k: v for k, v in Counter(df_grp_agg_attr["TypeName"]).most_common()}, index=["TypeName"])

,RS6 Avant quattro,SL 500,911 Turbo,RS6 Avant 5.0 V10 quattro,SL 600,SL 55 AMG,Cayenne Turbo,null,M5,A8 4.2 quattro,...,1300 VC Gordini,MX-5 1.8i 16V Confort,Turbo 16V,320d xDrive M-Sportpaket,Espace 2.0 T Privilège,1000,MP4-12C Cabriolet 3.8 V8,WLR,A 150 Classic,Reventon Coupé
TypeName,31,30,29,23,21,18,18,16,16,15,...,1,1,1,1,1,1,1,1,1,1


### Normalize city/zip

In [26]:
pd.DataFrame({k: v for k, v in Counter(df_grp_agg_attr["City"]).most_common()}, index=["City"])

,Zuzwil,Porrentruy,Sursee,Safenwil,Basel,St. Gallen
City,1118,14,11,5,3,2


In [27]:
pd.DataFrame({k: v for k, v in Counter(t_data["city"]).most_common()}, index=["city"])

,Zuzwil,De Lier,Bovenden,Aalter,London,NaN,BRUMMEN,Singen,Paris,Waalwijk,...,market harborough,St. Gallen,Sydney,Perth,Dortmund,97447 Gerolzhofen,Aubussargues,Milano,Dubai - United Arab Emirates,PADOVA
city,675,513,338,243,221,218,208,191,176,159,...,1,1,1,1,1,1,1,1,1,1


In [28]:
# Normalization
# Normalisation is required in case an attribute value is different but actually is the same (different
# spelling, language, different unit used etc.).
# 
# E.g. the first column in the target data "carType" defines cat Types (	Coupé	Convertible / Roadster	Other etc.)
# The column BodyTypeText in the input data can be used as carTypes for the target data 
# but it uses slightly different names (some are in German) and some are missgng (e.g. Single seater)
# We could use the number of seats to find the "Single seater" cars for the target data

# Normalize column "carType"
# it is neccessary to change the names of the car types found in BodyTypeText column of the input data to match
# the names used in Target Data carType column and use the number of seats to find the "Single seater". 
# If the BodyType in input data can't be assigned to one of the types in target data, put it to "Other"
def normalize_cartype(row):
    if row["Seats"] == '1':
        return "Single seater"
    try:
        return {
            "Coupé": "Coupé",
            "Limousine": "Saloon",
            "Cabriolet": "Convertible / Roadster",
            "Kombi": "Station Wagon",
            "SUV / Geländewagen": "SUV",
        }[row["BodyTypeText"]]
    except KeyError:
        return "Other"

# Normalize column "color"
# the same as with carType. Some colors are in German and some are slightly different. 
# Assign them to "standard" values from target data
def normalize_color(row):
    for color_en, color_de in [
            ("Black", ["schwarz"]), ("Silver", ["silber"]), ("Blue", ["blau"]), ("Gray", ["grau", "anthrazit"]), 
            ("White", ["weiss"]), ("Red", ["red", "bordeaux"]), ("Green", ["grün"]), ("Yellow", ["gelb"]), 
            ("Purple", ["violett"]), ("Gold", ["gold"]), ("Brown", ["braun"]), ("Orange", ["orange"]), ("Beige", ["beige"])]:
        for item in color_de:
            if item in row["BodyColorText"]:
                return color_en
    return "Other"

# Normalize column ConditionTypeText
def normalize_condition(row):
    try:
        return {
            "Occasion": "Used",
            "Oldtimer": "Restored",
            "Neu": "New",
            "Vorführmodell": "Original Condition",
        }[row["ConditionTypeText"]]
    except KeyError:
        return "Other"
    
# Normalize columns model and model_variant
# In order to extract  variant we can remove  model  from the column ModelTypeText. 
# If this doesn't work, we use TypeName.
def normalize_variant(row):
    model = row["ModelText"].strip()
    model_variant = row["ModelTypeText"].strip()
    if model_variant[:len(model)].lower() == model.lower():
        return model_variant[len(model):].strip()
    return row["TypeName"]

# Normalize column Zip
# Can be done automatically with e.g. pgeocode
def normalize_zip(row):
    try:
        return {
            "Zuzwil": "9524",
            "Porrentruy": "2900",
            "Sursee": "6210",
            "Safenwil": "5745",
            "Basel": "4000",
            "St. Galen": "9000",
        }[row["City"]]
    except KeyError:
        return "Other"

In [29]:
# Perform normalization of all columns posible to normalize
NORM_FUNCT = {
    "BodyTypeText": normalize_cartype,
    "BodyColorText": normalize_color,
    "Condition": normalize_condition,
    "Variant": normalize_variant,
    "Zip": normalize_zip,
}

def normalize(row):
    for column, funct in NORM_FUNCT.items():
        row[column] = funct(row)
    return row
    
# perform normalization    
normalized_df = df_grp_agg_attr.apply(normalize, axis=1)
normalized_df.head()

,BodyColorText,BodyTypeText,Ccm,City,Co2EmissionText,ConditionTypeText,ConsumptionRatingText,ConsumptionTotalText,Doors,DriveTypeText,...,ModelText,ModelTypeText,Properties,Seats,TransmissionTypeText,TypeName,TypeNameFull,Condition,Variant,Zip
0,Gray,Saloon,3199,Zuzwil,275 g/km,Occasion,null,11.5 l/100km,4,Allrad,...,E 320,E 320 Elégance 4-Matic,"""Ab MFK""",5,Automat,E 320 Elégance 4-Matic,MERCEDES-BENZ E 320 Elégance 4-Matic,Used,Elégance 4-Matic,9524
1,Gray,Station Wagon,4991,Zuzwil,333 g/km,Occasion,G,14.0 l/100km,5,Allrad,...,RS6,RS6 Avant 5.0 V10 quattro,"""Ab MFK""",5,Automat sequentiell,RS6 Avant 5.0 V10 quattro,AUDI RS6 Avant 5.0 V10 quattro,Used,Avant 5.0 V10 quattro,9524
2,Gray,Station Wagon,4172,Zuzwil,350 g/km,Occasion,G,14.6 l/100km,5,Allrad,...,RS6,RS6 Avant quattro,"""Ab MFK""",5,Automat sequentiell,RS6 Avant quattro,AUDI RS6 Avant quattro,Used,Avant quattro,9524
3,Gray,Coupé,6162,Zuzwil,291 g/km,Occasion,G,12.7 l/100km,2,Hinterradantrieb,...,CORVETTE,Corvette Z06,"""Ab MFK"", ""Direkt-/Parallelimport""",2,Schaltgetriebe manuell,Corvette Z06,CHEVROLET Corvette Z06,Used,Z06,9524
4,Gray,SUV,4806,Zuzwil,270 g/km,Occasion,G,11.5 l/100km,5,Allrad,...,CAYENNE,Cayenne Turbo Techart Magnum Kit,"""Ab MFK""",5,Automat sequentiell,Cayenne Turbo Techart Magnum Kit,PORSCHE Cayenne Turbo Techart Magnum Kit,Used,Turbo Techart Magnum Kit,9524


In [30]:
def integrate(row):
    return pd.Series({
        "carType": row["BodyTypeText"],
        "color": row["BodyColorText"],
        "condition": row["Condition"],
        "currency": "CHF", # assume that all cars are from/to be sold CH
        "drive": "LHD", # all cars in the input data are from CH, hence LHD, could not find column to normalize
        "city": row["City"], # all cities are from CH
        "country": "CH", # all cars in the input data are from CH but can be deduced from the city name
        "make": row["MakeText"],
        "manufacture_year": row["FirstRegYear"],
        "mileage": row["Km"], 
        "mileage_unit": "kilometer", # all cars are form CH
        "model": row["ModelText"],
        "model_variant": row["Variant"], 
        "price_on_request": None, # could not find column to normalize in the input data
        "type": "car", # all of target data contains the value "car"
        "zip": row["Zip"], # can be inferred from city using e.g pgeocode
        "manufacture_month": row["FirstRegMonth"],
        "fuel_consumption_unit": "l_km_consumption" if row["ConsumptionTotalText"] and row["ConsumptionTotalText"] != 'null' else None,
    })

# perform data integration
formated_df = normalized_df.apply(integrate, axis=1)
formated_df.head(20)

,carType,color,condition,currency,drive,city,country,make,manufacture_year,mileage,mileage_unit,model,model_variant,price_on_request,type,zip,manufacture_month,fuel_consumption_unit
0,Saloon,Gray,Used,CHF,LHD,Zuzwil,CH,MERCEDES-BENZ,1999,31900,kilometer,E 320,Elégance 4-Matic,None,car,9524,1,l_km_consumption
1,Station Wagon,Gray,Used,CHF,LHD,Zuzwil,CH,AUDI,2008,25400,kilometer,RS6,Avant 5.0 V10 quattro,None,car,9524,7,l_km_consumption
2,Station Wagon,Gray,Used,CHF,LHD,Zuzwil,CH,AUDI,2002,38500,kilometer,RS6,Avant quattro,None,car,9524,10,l_km_consumption
3,Coupé,Gray,Used,CHF,LHD,Zuzwil,CH,CHEVROLET,2015,200,kilometer,CORVETTE,Z06,None,car,9524,6,l_km_consumption
4,SUV,Gray,Used,CHF,LHD,Zuzwil,CH,PORSCHE,2010,2900,kilometer,CAYENNE,Turbo Techart Magnum Kit,None,car,9524,1,l_km_consumption
5,Coupé,Gray,Used,CHF,LHD,Zuzwil,CH,FORD (USA),2012,92000,kilometer,MUSTANG,Shelby GT500,None,car,9524,3,None
6,Saloon,Gray,Used,CHF,LHD,Zuzwil,CH,MERCEDES-BENZ,2004,120900,kilometer,CLS 500,,None,car,9524,11,l_km_consumption
7,Convertible / Roadster,Gray,Used,CHF,LHD,Zuzwil,CH,ASTON MARTIN,2005,31800,kilometer,DB9,Volante,None,car,9524,11,l_km_consumption
8,Convertible / Roadster,Gray,Used,CHF,LHD,Zuzwil,CH,LOTUS,2003,38200,kilometer,ELISE,,None,car,9524,10,l_km_consumption
9,Coupé,Gray,Restored,CHF,LHD,Zuzwil,CH,LAMBORGHINI,1973,48000,kilometer,nan,Espada GT 400 Serie 3,None,car,9524,4,None


In [31]:
t_data.head()

,carType,color,condition,currency,drive,city,country,make,manufacture_year,mileage,mileage_unit,model,model_variant,price_on_request,type,zip,manufacture_month,fuel_consumption_unit
0,Convertible / Roadster,White,Used,USD,LHD,Zuzwil,CH,Toyota,1983,0.0,kilometer,Celica,2800 i Supra,False,car,NaN,3.0,l_km_consumption
1,Convertible / Roadster,Other,Original Condition,USD,LHD,London,GB,Ferrari,2001,0.0,mile,550,NaN,True,car,NaN,NaN,NaN
2,Convertible / Roadster,White,Used,USD,LHD,Scotts Valley,US,Porsche,1989,0.0,mile,"911 ""G""",NaN,True,car,NaN,NaN,NaN
3,Convertible / Roadster,Blue,Used,USD,LHD,London,GB,Rolls-Royce,1961,0.0,kilometer,Silver Cloud II,NaN,False,car,NaN,NaN,NaN
4,Convertible / Roadster,Black,Original Condition,USD,LHD,Hong Kong,HK,Gumpert,2010,0.0,kilometer,Apollo,Apollo S 800hp,False,car,NaN,NaN,NaN


In [32]:
with pd.ExcelWriter("onedot_data_analyst_solution.xlsx") as writer:
    df_grp_agg_attr.to_excel(writer, sheet_name="Pre-processed Data", index=False, na_rep="null")
    normalized_df.to_excel(writer, sheet_name="Normalized Data", index=False, na_rep="null")
    formated_df.to_excel(writer, sheet_name="Integrated Data", index=False, na_rep="null")

In [33]:
pd.DataFrame({k: v for k, v in Counter(normalized_df["BodyTypeText"]).most_common()}, index=["BodyTypeText"])

,Coupé,Saloon,Convertible / Roadster,Station Wagon,SUV,Other,Single seater
BodyTypeText,367,325,248,98,89,24,2
